In [1]:
import random
import string
import time
import base64
from hashlib import sha256
from hmac import HMAC

import http.client
import requests
import pandas as pd 

APPID = "tubmafwrzhpgfiuf"
SECRET = "eotpcqbvhycdshwscqnytiwzbgonposs"


def generate_nonce(length=32):
    """Generate a random nonce."""
    return ''.join(random.choices(string.ascii_letters + string.digits, k=length))


def get_timestamp():
    """Get the current timestamp."""
    return int(time.time())


def build_sign_str(appid, nonce, timestamp):
    """Build the string to be signed."""
    return f'appid={appid}&nonce={nonce}&timestamp={timestamp}'


def calculate_signature(secret, message):
    """Calculate the HMAC SHA-256 signature."""
    return base64.urlsafe_b64encode(HMAC(secret.encode('utf-8'), message.encode('utf-8'), sha256).digest()).decode('utf-8')


def fetch_indicator_details(indicator_id):
    """Fetch the details for a specific indicator ID."""
    nonce = generate_nonce()
    timestamp = get_timestamp()
    sign_str = build_sign_str(APPID, nonce, timestamp)
    signature = calculate_signature(SECRET, sign_str)

    headers = {
        'nonce': nonce,
        'timestamp': str(timestamp),
        'appid': APPID,
        'signature': signature,
        'Accept': "*/*",
        'Accept-Encoding': "gzip, deflate, br",
        'User-Agent': "PostmanRuntime-ApipostRuntime/1.1.0",
        'Connection': "keep-alive",
    }

    url = f"https://etahub.hzinsights.com/v1/edb/data?EdbCode={indicator_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json().get('Data')
    else:
        print(
            f"Failed to fetch data for ID {indicator_id}, status code: {response.status_code}")
        return None

def fetch_indicator_name(indicator_id):
    """Fetch the name for a specific indicator ID."""
    nonce = generate_nonce()
    timestamp = get_timestamp()
    sign_str = build_sign_str(APPID, nonce, timestamp)
    signature = calculate_signature(SECRET, sign_str)

    headers = {
        'nonce': nonce,
        'timestamp': str(timestamp),
        'appid': APPID,
        'signature': signature,
        'Accept': "*/*",
        'Accept-Encoding': "gzip, deflate, br",
        'User-Agent': "PostmanRuntime-ApipostRuntime/1.1.0",
        'Connection': "keep-alive",
    }

    url = f"https://etahub.hzinsights.com/v1/edb/detail?EdbCode={indicator_id}"
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json().get('Data').get('EdbName')
    else:
        print(
            f"Failed to fetch data for ID {indicator_id}, status code: {response.status_code}")
        return None

def main():
    # List of indicator IDs you want to fetch
    indicator_ids = ['Cstl2502251012427771',"C2501061643218860", 'C2303251397','C2311290091','C2108254891_250225160109',
                    'Cstl2502251357196896_250225153903','Cstl2502251358028925_250225154445','Cstl2502181513479083_250225154231','Cstl2502191524558885_250225154644',
                    'C2201191346_250225154942','a12726346_250225155342','C2411181446305731_250224165212','ID01542379_250225155546']  # Add more IDs as needed

    # Dictionary to store DataFrames for each indicator
    data_frames = {}

    for indicator_id in indicator_ids:
        data = fetch_indicator_details(indicator_id)
        if data:
            # Create a DataFrame with DataTime as index
            df = pd.DataFrame(data)
            df['DataTime'] = pd.to_datetime(df['DataTime'])
            df.set_index('DataTime', inplace=True)
            df.sort_index(inplace=True)
            # Only keep the 'Value' column and rename it to the indicator ID
            df = df[['Value']].rename(columns={'Value': fetch_indicator_name(indicator_id)})
            data_frames[indicator_id] = df

    # Concatenate all DataFrames along the columns
    if data_frames:
        result_df = pd.concat(data_frames.values(), axis=1)
        print(result_df.info())
        result_df.to_excel("data_input_auto/中国柴油实际消费量Non-Trend-F0.2.xlsx")
        print("Data saved successfully as '中国柴油实际消费量Non-Trend-F0.2.xlsx'")



if __name__ == "__main__":
    main()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6381 entries, 2007-12-14 to 2026-05-12
Data columns (total 13 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   中国柴油实际消费量Non-Trend/F0.2               262 non-null    float64
 1   中国10年期国债收益率Non-Trend/F0.1             4229 non-null   float64
 2   DR007利率/15DMA-1年期MLF利率                2346 non-null   float64
 3   中国期限利差-美国期限利差                         4210 non-null   float64
 4   30大中城市商品房成交面积/7DMA(预测/上一年季节性)         5872 non-null   float64
 5   周度螺纹表需Non-Trend/F0.2(预测/上一年季节性)       460 non-null    float64
 6   水泥表观消费Non-Trend/F0.2(预测/上一年季节性)       267 non-null    float64
 7   每日建材成交/5DMANon-Trend/F0.1(预测/上一年季节性)  2449 non-null   float64
 8   五大钢材周度表需Non-Trend/F0.1(预测/上一年季节性)     460 non-null    float64
 9   铁矿港口成交量/5DMA(预测/上一年季节性)               1770 non-null   float64
 10  SMM中国电线电缆周度开工率(预测/上一年季节性)             74 non-null     float64
 11 

Data saved successfully as '中国柴油实际消费量Non-Trend-F0.2.xlsx'
